# Behavior Classification using Explainable Active Learning Model

## 1. Load Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

import os
from dotenv import load_dotenv
load_dotenv()
DLC_DATASET_PATH = os.getenv('DLC_DATASET_PATH')

In [2]:
FRAME_RATE = 30
SAMPLE_RATE = 10
MULTI_ANIMAL = False
LIKELIHOOD_THRESHOLD = 0.6
BODY_PARTS = ["tailbase", "earR", "earL", "msBase", "msTop", "centroid", "cleft", "cright"]
RULE_BASED_LABELS = ["shuttles_label_naive1", "shuttles_label_naive2", "shuttles_label_naive3", "shuttles_label_hardcoded", "freezing_label"]
labels = RULE_BASED_LABELS+["other"]

In [3]:
# Multi-level header
df_pose_labels = pd.read_csv(DLC_DATASET_PATH + "/labelled_DLC.csv", header=[0, 1], index_col=0)

# Flatten the multi-level header into single strings
df_pose_labels.columns = ['_'.join(col).strip() for col in df_pose_labels.columns.values]

# Remove second level header if it is empty
df_pose_labels.columns = [col.split("_Unnamed")[0] if "_Unnamed" in col else col for col in df_pose_labels.columns.values]

# Remove NaN columns and rows
df_pose_labels = df_pose_labels.dropna(axis=1, how='all')
df_pose_labels = df_pose_labels.dropna(axis=0, how='any')

print("Shape of df_pose_labels:", df_pose_labels.shape)
df_pose_labels.head()

Shape of df_pose_labels: (20000, 32)


,tailbase_x,tailbase_y,tailbase_likelihood,earR_x,earR_y,earR_likelihood,earL_x,earL_y,earL_likelihood,msBase_x,...,cright_y,cright_likelihood,calc_centroid_x,calc_centroid_y,speed,shuttles_label_naive1,shuttles_label_naive2,shuttles_label_naive3,shuttles_label_hardcoded,freezing_label
0,215.430298,318.572968,0.999996,335.296844,346.048737,0.999918,351.505981,309.215546,0.999891,351.370636,...,345.997833,0.999949,313.400940,326.633553,0.000000,0,0,0,0,0
1,216.124664,319.105774,0.999997,334.252045,349.912659,0.999950,351.527374,307.669678,0.999980,351.550171,...,347.752991,0.999950,313.363564,327.412323,0.779667,0,0,0,0,0
2,215.643936,319.362427,0.999998,333.880310,349.696228,0.999942,351.895233,308.665619,0.999976,352.242249,...,348.114502,0.999924,313.415432,327.838448,0.429270,0,0,0,0,0
3,216.000183,319.383545,0.999997,334.263062,349.323456,0.999920,351.224792,307.465485,0.999981,351.896088,...,347.783112,0.999937,313.346031,327.274757,0.567946,0,0,0,0,0
4,215.784775,319.112488,0.999998,334.187714,349.681763,0.999959,351.621399,307.867065,0.999985,351.576569,...,347.832947,0.999970,313.292614,327.455528,0.188498,0,0,0,0,0


In [4]:
df_labels = df_pose_labels[RULE_BASED_LABELS]

print("Shape of df_labels:", df_labels.shape)
df_labels.head()

Shape of df_labels: (20000, 5)


,shuttles_label_naive1,shuttles_label_naive2,shuttles_label_naive3,shuttles_label_hardcoded,freezing_label
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [5]:
# all unlabeled are placed into "other"
df_labels_other = df_labels.copy()
df_labels_other["other"] = 0

# find all unlabeled
unlabeled_data = df_labels_other.sum(axis=1) == 0

# change all unlabeled to 1 in other
df_labels_other.loc[unlabeled_data, "other"] = 1

print("Shape of df_labels_other:", df_labels_other.shape)
df_labels_other.head()

Shape of df_labels_other: (20000, 6)


,shuttles_label_naive1,shuttles_label_naive2,shuttles_label_naive3,shuttles_label_hardcoded,freezing_label,other
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [6]:
df_pose = df_pose_labels.drop(columns=RULE_BASED_LABELS)

print("Shape of df_pose:", df_pose.shape)
df_pose.head()

Shape of df_pose: (20000, 27)


,tailbase_x,tailbase_y,tailbase_likelihood,earR_x,earR_y,earR_likelihood,earL_x,earL_y,earL_likelihood,msBase_x,...,centroid_likelihood,cleft_x,cleft_y,cleft_likelihood,cright_x,cright_y,cright_likelihood,calc_centroid_x,calc_centroid_y,speed
0,215.430298,318.572968,0.999996,335.296844,346.048737,0.999918,351.505981,309.215546,0.999891,351.370636,...,0.999951,281.177429,278.164520,0.999803,275.792725,345.997833,0.999949,313.400940,326.633553,0.000000
1,216.124664,319.105774,0.999997,334.252045,349.912659,0.999950,351.527374,307.669678,0.999980,351.550171,...,0.999983,281.244598,276.559937,0.999884,277.471069,347.752991,0.999950,313.363564,327.412323,0.779667
2,215.643936,319.362427,0.999998,333.880310,349.696228,0.999942,351.895233,308.665619,0.999976,352.242249,...,0.999982,282.179657,276.486542,0.999849,276.880859,348.114502,0.999924,313.415432,327.838448,0.429270
3,216.000183,319.383545,0.999997,334.263062,349.323456,0.999920,351.224792,307.465485,0.999981,351.896088,...,0.999978,283.015259,276.601379,0.999904,277.634735,347.783112,0.999937,313.346031,327.274757,0.567946
4,215.784775,319.112488,0.999998,334.187714,349.681763,0.999959,351.621399,307.867065,0.999985,351.576569,...,0.999984,282.534119,277.098724,0.999917,276.700134,347.832947,0.999970,313.292614,327.455528,0.188498


In [7]:
df_pose_labels_other = pd.concat([df_pose, df_labels_other], axis=1)

print("Shape of df_pose_labels_other:", df_pose_labels_other.shape)
df_pose_labels_other.head()

Shape of df_pose_labels_other: (20000, 33)


,tailbase_x,tailbase_y,tailbase_likelihood,earR_x,earR_y,earR_likelihood,earL_x,earL_y,earL_likelihood,msBase_x,...,cright_likelihood,calc_centroid_x,calc_centroid_y,speed,shuttles_label_naive1,shuttles_label_naive2,shuttles_label_naive3,shuttles_label_hardcoded,freezing_label,other
0,215.430298,318.572968,0.999996,335.296844,346.048737,0.999918,351.505981,309.215546,0.999891,351.370636,...,0.999949,313.400940,326.633553,0.000000,0,0,0,0,0,1
1,216.124664,319.105774,0.999997,334.252045,349.912659,0.999950,351.527374,307.669678,0.999980,351.550171,...,0.999950,313.363564,327.412323,0.779667,0,0,0,0,0,1
2,215.643936,319.362427,0.999998,333.880310,349.696228,0.999942,351.895233,308.665619,0.999976,352.242249,...,0.999924,313.415432,327.838448,0.429270,0,0,0,0,0,1
3,216.000183,319.383545,0.999997,334.263062,349.323456,0.999920,351.224792,307.465485,0.999981,351.896088,...,0.999937,313.346031,327.274757,0.567946,0,0,0,0,0,1
4,215.784775,319.112488,0.999998,334.187714,349.681763,0.999959,351.621399,307.867065,0.999985,351.576569,...,0.999970,313.292614,327.455528,0.188498,0,0,0,0,0,1


## 2. Preprocessing using A-SOiD Pipeline

### 2.1 Filter by Likelihood

Smooths out unreliable keypoint coordinates based on confidence values (likelihoods). This is a "hold last good value" filter to avoid jittery or missing points.

The `adp_filt` function is rewrite from the `adp_filt` function in A-SOiD (likelihood adaptive filtering).

In [8]:
# likelihood adaptive filtering
def adp_filt(df_pose_labels_other, idx_coord, idx_llh, llh_value, labels):
    """
    For body parts with likelihood values below the threshold, copy the previous valid row's x, y coordinates.
    Labels are not modified during filtering and are set back after filtering.

    Parameters:
        df_pose_labels_other (pd.DataFrame): The input DataFrame containing x, y, likelihood, and pose labels.
        idx_coord (list): Indices of selected body parts (x and y columns).
        idx_llh (list): Indices of likelihood columns.
        llh_value (float): Threshold for likelihood filtering.
        labels (list): List of pose labels to retain in the output.

    Returns:
        pd.DataFrame: DataFrame with invalid x, y coordinates corrected.
        dict: Statistics on filtered body parts and likelihood values below the threshold.
    """
    # Convert DataFrame to numpy arrays for x, y, likelihood, and pose labels
    data_x_coord = np.array(df_pose_labels_other.iloc[:, idx_coord[::2]])
    data_y_coord = np.array(df_pose_labels_other.iloc[:, idx_coord[1::2]])
    data_llh = np.array(df_pose_labels_other.iloc[:, idx_llh])
    original_labels = df_pose_labels_other[labels].copy()  # Preserve original labels

    # Initialize statistics
    llh_below_threshold = 0
    total_llh = data_llh.size  # Total number of likelihood values
    body_part_stats = {df_pose_labels_other.columns[idx_coord[::2][i]]: 0 for i in range(len(idx_coord[::2]))}

    # Iterate through body parts and correct invalid x, y coordinates
    for x in range(data_llh.shape[1]):  # Iterate over each body part
        for i in range(1, data_llh.shape[0]):  # Start from the second row
            if data_llh[i, x] < llh_value:  # If likelihood is below the threshold
                llh_below_threshold += 1
                body_part_stats[df_pose_labels_other.columns[idx_coord[::2][x]]] += 1
                # Copy the previous row's x, y coordinates for this body part
                data_x_coord[i, x] = data_x_coord[i - 1, x]
                data_y_coord[i, x] = data_y_coord[i - 1, x]

    # Replace the x, y columns in the DataFrame with corrected values
    for idx, col_idx in enumerate(idx_coord[::2]):  # Update x columns
        df_pose_labels_other.iloc[:, col_idx] = data_x_coord[:, idx]
    for idx, col_idx in enumerate(idx_coord[1::2]):  # Update y columns
        df_pose_labels_other.iloc[:, col_idx] = data_y_coord[:, idx]

    # Restore the original labels
    df_pose_labels_other[labels] = original_labels

    # Calculate likelihood ratio
    llh_ratio = llh_below_threshold / total_llh

    # Prepare statistics
    stats = {
        "llh_below_threshold": llh_below_threshold,
        "total_llh": total_llh,
        "llh_below_threshold_ratio": llh_ratio,
        "body_part_stats": body_part_stats
    }

    return df_pose_labels_other, stats

In [9]:
# Get numeric indices of columns ending with "_likelihood"
idx_llh = [i for i, col in enumerate(df_pose_labels_other.columns) if col.endswith("_likelihood")]
labels = RULE_BASED_LABELS+["other"]
# Get numeric indices of columns of x and y coordinates
idx_coord = [i for i, col in enumerate(df_pose_labels_other.columns) if not col.endswith("_likelihood") and col not in labels]

filt_df_pose_labels_other, stats = adp_filt(
    df_pose_labels_other=df_pose_labels_other,
    idx_coord=idx_coord,
    idx_llh=idx_llh,
    llh_value=LIKELIHOOD_THRESHOLD,
    labels=labels
)

print("Filtered DataFrame's shape:")
print(filt_df_pose_labels_other.shape)
print("\nStatistics:")
print(stats)

Filtered DataFrame's shape:
(20000, 33)

Statistics:
{'llh_below_threshold': 1658, 'total_llh': 160000, 'llh_below_threshold_ratio': 0.0103625, 'body_part_stats': {'tailbase_x': 287, 'earR_x': 261, 'earL_x': 258, 'msBase_x': 279, 'msTop_x': 284, 'centroid_x': 14, 'cleft_x': 158, 'cright_x': 117, 'calc_centroid_x': 0, 'speed': 0}}


In [10]:
filt_df_pose = filt_df_pose_labels_other.drop(columns=RULE_BASED_LABELS + ["other"])

print("Filtered DataFrame's shape without labels:")
print(filt_df_pose.shape)
filt_df_pose.head()

Filtered DataFrame's shape without labels:
(20000, 27)


,tailbase_x,tailbase_y,tailbase_likelihood,earR_x,earR_y,earR_likelihood,earL_x,earL_y,earL_likelihood,msBase_x,...,centroid_likelihood,cleft_x,cleft_y,cleft_likelihood,cright_x,cright_y,cright_likelihood,calc_centroid_x,calc_centroid_y,speed
0,215.430298,318.572968,0.999996,335.296844,346.048737,0.999918,351.505981,309.215546,0.999891,351.370636,...,0.999951,281.177429,278.164520,0.999803,275.792725,345.997833,0.999949,313.400940,326.633553,0.000000
1,216.124664,319.105774,0.999997,334.252045,349.912659,0.999950,351.527374,307.669678,0.999980,351.550171,...,0.999983,281.244598,276.559937,0.999884,277.471069,347.752991,0.999950,313.363564,327.412323,0.779667
2,215.643936,319.362427,0.999998,333.880310,349.696228,0.999942,351.895233,308.665619,0.999976,352.242249,...,0.999982,282.179657,276.486542,0.999849,276.880859,348.114502,0.999924,313.415432,327.838448,0.429270
3,216.000183,319.383545,0.999997,334.263062,349.323456,0.999920,351.224792,307.465485,0.999981,351.896088,...,0.999978,283.015259,276.601379,0.999904,277.634735,347.783112,0.999937,313.346031,327.274757,0.567946
4,215.784775,319.112488,0.999998,334.187714,349.681763,0.999959,351.621399,307.867065,0.999985,351.576569,...,0.999984,282.534119,277.098724,0.999917,276.700134,347.832947,0.999970,313.292614,327.455528,0.188498


In [11]:
filt_df_labels_other = filt_df_pose_labels_other[RULE_BASED_LABELS + ["other"]]

print("Filtered DataFrame's shape with labels:")
print(filt_df_labels_other.shape)
filt_df_labels_other.head()

Filtered DataFrame's shape with labels:
(20000, 6)


,shuttles_label_naive1,shuttles_label_naive2,shuttles_label_naive3,shuttles_label_hardcoded,freezing_label,other
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


## 3. Feature Engineering

## 4. Rule-based Labeling (skipped for now)

## 5. Semi-supervised metric learning → low-dimensional vector embedding space

## 6. Clustering (active learning loop skipped for now)

## 7. Train a decision tree classification model on engineered features to predict cluster IDs. This way we can get explainable feature importance (Explainability Model, interpretable)

## 8. Train a classification model (maybe also decision tree) on embedded (maybe: and clustered) data to classify behaviors (Performance Model, accurate but abstract)